<a href="https://colab.research.google.com/github/Dotty1004/Sample/blob/main/%EC%9D%8C%EC%84%B1%EB%B6%84%EC%84%9D%EA%B4%80.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
# -------------------------------------------------------------------------
# 1. 필요한 라이브러리 설치 (GPU 환경에 최적화)
# -------------------------------------------------------------------------
# %%capture는 설치 과정의 긴 메시지를 숨겨 깔끔하게 보여줍니다.
%%capture
!pip install faster-whisper
!pip install librosa pydub

# -------------------------------------------------------------------------
# 2. 필요한 기능들 불러오기
# -------------------------------------------------------------------------
import os
import numpy as np
import librosa
from faster_whisper import WhisperModel
import torch
from google.colab import files
from IPython.display import display, HTML
import base64
from google.colab import output

# -------------------------------------------------------------------------
# 3. 음성 분석을 위한 핵심 클래스 정의
# -------------------------------------------------------------------------
class VoiceFeatureExtractor:
    def __init__(self, model_size="medium"):
        """
        분석기 초기화. 오프라인 음성 인식 모델을 로드합니다.
        GPU를 사용하므로 'medium' 모델을 사용하여 정확도를 높입니다.
        """
        print("="*50)
        print("1. 음성 인식 모델을 로드하는 중입니다...")
        # GPU 사용 설정: device="cuda", compute_type="float16"
        self.model = WhisperModel(model_size, device="cuda", compute_type="float16")
        print("   모델 로드 완료. (모델: {}, 장치: GPU)".format(model_size))
        print("="*50)

    def analyze(self, audio_file_path):
        """
        오디오 파일을 받아 전체 분석을 수행하고 결과를 텍스트로 반환합니다.
        """
        print(f"\n2. '{os.path.basename(audio_file_path)}' 파일 분석을 시작합니다...")

        # (1) 음성 파일 로드 및 기본 정보 추출
        y, sr = librosa.load(audio_file_path, sr=16000)
        duration = librosa.get_duration(y=y, sr=sr)

        # (2) 음성 -> 텍스트 변환 (STT) with faster-whisper on GPU
        segments, _ = self.model.transcribe(audio_file_path, word_timestamps=True)

        words_with_timestamps = []
        full_text = ""
        for segment in segments:
            full_text += segment.text
            for word in segment.words:
                words_with_timestamps.append({'word': word.word.strip().replace(",", "").replace(".", ""), 'start': word.start, 'end': word.end})

        print("   - 음성을 텍스트로 변환 완료.")

        # (3) 음성 특징 추출
        word_count = len(words_with_timestamps)
        speech_rate_wpm = (word_count / duration) * 60 if duration > 0 else 0

        # 음높이(Pitch) 분석
        pitches, magnitudes = librosa.piptrack(y=y, sr=sr)
        valid_pitches = pitches[magnitudes > np.median(magnitudes)]
        pitch_variation_std = np.std(valid_pitches) if len(valid_pitches) > 0 else 0.0

        print("   - 목소리 특징 추출 완료.")

        # (4) 군말(Filler Words) 분석
        filler_words_list = ['아', '음', '그', '이제', '막', '뭐랄까', '사실', '약간', '저기', '근데', '뭔가']
        filler_count = 0
        found_fillers = []
        for word_info in words_with_timestamps:
            if word_info['word'] in filler_words_list:
                filler_count += 1
                found_fillers.append(word_info['word'])

        filler_ratio = (filler_count / word_count) * 100 if word_count > 0 else 0

        print("   - 언어 습관 분석 완료.")
        print("\n3. 모든 분석이 완료되었습니다.")
        print("="*50)


        # (5) 결과를 GPT에게 전달할 형식으로 정리
        report = f"""
### [음성 분석 데이터]
- **전체 발화 내용:** "{full_text}"
- **음성 길이:** {duration:.2f}초
- **발화 속도:** 분당 약 {speech_rate_wpm:.0f} 단어 (WPM)
- **억양 변화 수준:** {pitch_variation_std:.2f} (수치가 클수록 다채롭고, 25 미만이면 단조로울 수 있음)
- **군말 사용 빈도:** 총 {filler_count}회 사용 (전체 단어의 {filler_ratio:.1f}%)
- **주요 군말 리스트:** {', '.join(list(set(found_fillers)))}
"""
        return report

# -------------------------------------------------------------------------
# 4. 실제 프로그램 실행 부분
# -------------------------------------------------------------------------
def run_analysis_on_upload(file_contents, filename):
    # (1) 분석기 생성
    extractor = VoiceFeatureExtractor(model_size="medium")

    # (2) 임시 파일로 저장
    with open(filename, "wb") as f:
        f.write(file_contents)

    # (3) 분석 실행 및 결과 출력
    analysis_report = extractor.analyze(filename)

    print("\n\n✅ 분석 데이터가 생성되었습니다! ✅")
    print("아래 [음성 분석 데이터]부터 마지막 줄까지 모두 복사하여,")
    print("준비된 GPTs 또는 Gemini 맞춤형 챗봇에 붙여넣고 피드백을 요청하세요.\n")
    print("--- 여기부터 복사하세요 ---")
    print(analysis_report.strip())
    print("--- 여기까지 복사하세요 ---")

    # 임시 파일 삭제
    os.remove(filename)

def upload_and_run():
    print("\n분석할 MP3, M4A, WAV 파일을 업로드 해주세요.")
    upload_button = """
    <input type="file" id="file_upload">
    <script>
      const fileUpload = document.getElementById('file_upload');
      fileUpload.addEventListener('change', (event) => {
        const file = event.target.files[0];
        const reader = new FileReader();
        reader.onload = (e) => {
          const b64_data = e.target.result.split(',')[1];
          google.colab.kernel.invokeFunction('notebook.run_analysis_on_upload', [b64_data, file.name], {});
        };
        reader.readAsDataURL(file);
      });
    </script>
    """
    display(HTML(upload_button))

    # Register the Python function so it can be called from Javascript
    output.register_callback('notebook.run_analysis_on_upload', run_analysis_on_upload_wrapper)

def run_analysis_on_upload_wrapper(b64_data, filename):
    file_contents = base64.b64decode(b64_data)
    run_analysis_on_upload(file_contents, filename)

# -------------------------------------------------------------------------
# 프로그램 실행
# -------------------------------------------------------------------------
upload_and_run()

In [7]:
upload_and_run()


분석할 MP3, M4A, WAV 파일을 업로드 해주세요.


1. 음성 인식 모델을 로드하는 중입니다...
   모델 로드 완료. (모델: medium, 장치: GPU)

2. '쿠팡마케팅.m4a' 파일 분석을 시작합니다...


/tmp/ipython-input-3178164168.py:40: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(audio_file_path, sr=16000)
/usr/local/lib/python3.11/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


   - 음성을 텍스트로 변환 완료.
   - 목소리 특징 추출 완료.
   - 언어 습관 분석 완료.

3. 모든 분석이 완료되었습니다.


✅ 분석 데이터가 생성되었습니다! ✅
아래 [음성 분석 데이터]부터 마지막 줄까지 모두 복사하여,
준비된 GPTs 또는 Gemini 맞춤형 챗봇에 붙여넣고 피드백을 요청하세요.

--- 여기부터 복사하세요 ---
### [음성 분석 데이터]
- **전체 발화 내용:** " 안녕하세요. 저는 도로시입니다. 모 대학 경영학과를 졸업하고 3년간 디지털 마케팅 분야에서 경험을 쌓아왔습니다. 특히 네이버 쇼핑과 카카오톡 채널을 활용한 퍼포먼스 마케팅 캠페인을 통해서 월 매출 30% 증가라는 성과를 달성하며 데이터 기반 마케팅 역량을 기룰 수 있었고 이 과정에서 고객 여정 분석과 개인화 마케팅의 중요성을 깨달았습니다. 앞으로는 쿠팡의 마케팅 기획팀에서 제가 가진 퍼포먼스 마케팅 경험과 데이터 분석 역량을 활용하여 고객 획득과 리텐션 향상에 기여하고 싶습니다."
- **음성 길이:** 38.99초
- **발화 속도:** 분당 약 100 단어 (WPM)
- **억양 변화 수준:** 889.00 (수치가 클수록 다채롭고, 25 미만이면 단조로울 수 있음)
- **군말 사용 빈도:** 총 0회 사용 (전체 단어의 0.0%)
- **주요 군말 리스트:**
--- 여기까지 복사하세요 ---
